In [1]:
!pip install tmdbsimple

In [2]:
import tmdbsimple as tmdb
import pandas as pd
import sqlite3 as sql
from google.cloud import bigquery
client = bigquery.Client()
connection = sql.connect(':memory:')

Using Kaggle's public dataset BigQuery integration.


In [3]:
tmdb.API_KEY = '637ae0caefc579558e30ed409fb20532'
search = tmdb.Search()
discover = tmdb.Discover()

In [4]:
watched = pd.read_csv('/kaggle/input/letterboxd/diary.csv')
reviewed = pd.read_csv('/kaggle/input/reviewed/reviews.csv')
liked_films = pd.read_csv('/kaggle/input/liked-films/films.csv')

In [5]:
watched

,Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date
0,2020-06-27,Parasite,2019,https://boxd.it/1dGmIN,4.5,NaN,NaN,2020-06-26
1,2020-06-27,Get Out,2017,https://boxd.it/1dGmRZ,4.5,NaN,NaN,2020-06-26
2,2020-06-27,Spirited Away,2001,https://boxd.it/1dGn79,3.5,NaN,NaN,2020-06-26
3,2020-06-27,Us,2019,https://boxd.it/1dGnht,4.5,NaN,NaN,2020-06-26
4,2020-06-27,Spider-Man: Homecoming,2017,https://boxd.it/1dGnof,3.0,NaN,NaN,2020-06-26
...,...,...,...,...,...,...,...,...
196,2024-07-12,Longlegs,2024,https://boxd.it/6Rakn5,4.0,NaN,NaN,2024-07-11
197,2024-07-16,Naruto 20th Anniversary - Road of Naruto,2022,https://boxd.it/6T5lcF,5.0,NaN,NaN,2024-07-15
198,2024-07-17,Boruto: Naruto the Movie,2015,https://boxd.it/6TDLft,4.5,Yes,NaN,2024-07-17
199,2024-07-25,Twisters,2024,https://boxd.it/6XeoPT,5.0,NaN,NaN,2024-07-24


In [6]:
reviewed

,Date,Content
0,2020-10-17,https://boxd.it/12IDQT
1,2020-12-22,https://boxd.it/1wDGWb
2,2020-12-22,https://boxd.it/1wDFOv
3,2020-12-31,https://boxd.it/1xVF0F
4,2021-02-08,https://boxd.it/1E8y5t
...,...,...
146,2024-05-25,https://boxd.it/6xw5DB
147,2024-06-29,https://boxd.it/6LB859
148,2024-06-29,https://boxd.it/6LBmpd
149,2024-07-12,https://boxd.it/6Raf7L


In [7]:
# Extract title for each film into a list

titles = watched.Name.tolist()
print(titles)
print(len(titles))

['Parasite', 'Get Out', 'Spirited Away', 'Us', 'Spider-Man: Homecoming', 'Inside Out', 'Coco', 'Harry Potter and the Prisoner of Azkaban', 'ERASED', 'Rascal Does Not Dream of Bunny Girl Senpai', 'DARLING in the FRANXX', 'Your Name.', 'Wolf Children', 'Whisper of the Heart', 'Hotarubi no Mori e', 'Rascal Does Not Dream of a Dreaming Girl', 'Demon Slayer: Kimetsu no Yaiba', 'When They See Us', 'Hope', 'Miracle in Cell No. 7', 'The Great Gatsby', 'The Greatest Showman', 'Coraline', 'The Devil All the Time', 'The Kissing Booth', 'Marriage Story', 'The Twilight Saga: Breaking Dawn – Part 1', 'Hereditary', 'The Hate U Give', 'The Silence of the Lambs', 'Hubie Halloween', 'Zodiac', 'Dream/Killer', 'After We Collided', 'The Imitation Game', 'The Social Network', 'Argo', 'The Amazing Spider-Man', 'The Amazing Spider-Man 2', 'Hacksaw Ridge', 'Snowden', 'Now You See Me 2', 'The Big Short', 'ariana grande: excuse me, i love you', 'Little Big Women', 'Hospital', 'The Trial of the Chicago 7', 'Fear 

In [8]:
# Extract release year of each film into a list
release_years = watched.Year.tolist()
print(release_years)
print(len(release_years))

[2019, 2017, 2001, 2019, 2017, 2015, 2017, 2004, 2016, 2018, 2018, 2016, 2012, 1995, 2011, 2019, 2019, 2019, 2013, 2013, 2013, 2017, 2009, 2020, 2018, 2019, 2011, 2018, 2018, 1991, 2020, 2007, 2015, 2020, 2014, 2010, 2012, 2012, 2014, 2016, 2016, 2016, 2015, 2020, 2020, 2020, 2020, 2021, 1999, 2019, 2012, 2008, 2013, 2017, 2012, 2018, 2019, 2019, 2021, 2016, 2018, 2019, 2018, 1996, 2003, 2005, 2014, 2016, 2010, 2017, 1999, 2012, 2009, 2021, 2021, 2021, 2021, 2004, 2007, 2021, 2019, 2007, 2011, 2017, 2022, 2020, 2021, 2012, 2007, 2018, 1997, 1997, 2011, 2015, 2022, 2005, 2013, 2001, 2004, 2009, 2013, 2017, 2022, 2014, 2021, 2015, 1975, 2022, 2022, 2014, 2018, 2016, 2009, 2021, 2020, 2019, 2014, 2016, 2010, 2004, 2013, 2022, 2016, 2011, 2006, 2019, 2022, 2018, 2018, 2005, 2022, 2007, 2005, 2009, 2011, 2010, 2022, 2022, 2022, 2007, 1999, 2010, 2019, 1996, 2009, 2022, 2018, 2013, 1994, 2023, 2023, 2004, 2023, 2023, 2023, 2004, 2017, 2022, 2019, 2018, 2016, 2023, 2018, 2023, 2023, 2006, 201

In [9]:
# Use tmdb.Search to get their ID
total_films = 0

for film, year in zip(titles, release_years):
    response = search.movie(query = film, year = year)
    for value in response['results']:
        if value['title'] == film:
            print(value['title']," / ",
                 value['release_date']," / ",
                 value['id'])
            
            total_films += 1    
            break
            
print("Total Number of Films: ", total_films)

Parasite  /  2019-05-30  /  496243
Get Out  /  2017-02-24  /  419430
Spirited Away  /  2001-07-20  /  129
Us  /  2019-03-14  /  458723
Spider-Man: Homecoming  /  2017-07-05  /  315635
Inside Out  /  2015-06-17  /  150540
Coco  /  2017-10-27  /  354912
Harry Potter and the Prisoner of Azkaban  /  2004-05-31  /  673
Your Name.  /  2016-08-26  /  372058
Wolf Children  /  2012-07-21  /  110420
Whisper of the Heart  /  1995-07-15  /  37797
Hotarubi no Mori e  /  2011-09-17  /  92321
Rascal Does Not Dream of a Dreaming Girl  /  2019-06-15  /  572154
Hope  /  2013-10-02  /  255709
Miracle in Cell No. 7  /  2013-01-23  /  158445
The Great Gatsby  /  2013-05-09  /  64682
The Greatest Showman  /  2017-12-20  /  316029
Coraline  /  2009-02-05  /  14836
The Devil All the Time  /  2020-09-11  /  499932
The Kissing Booth  /  2018-05-11  /  454983
Marriage Story  /  2019-09-28  /  492188
Hereditary  /  2018-06-07  /  493922
The Hate U Give  /  2018-10-19  /  470044
The Silence of the Lambs  /  1991-0

In [10]:
movie_id = []
result_count = 0

for film, year in zip(titles, release_years):
    response = search.movie(query = film, year = year)
    for value in response['results']:
        if value['title'] == film:
            movie_id.append(value['id'])
            result_count += 1
            break
print(movie_id)
print("Total Number of Films with ID: ", result_count)

[496243, 419430, 129, 458723, 315635, 150540, 354912, 673, 372058, 110420, 37797, 92321, 572154, 255709, 158445, 64682, 316029, 14836, 499932, 454983, 492188, 493922, 470044, 274, 617505, 1949, 333099, 613504, 205596, 37799, 68734, 1930, 102382, 324786, 302401, 291805, 318846, 774372, 739269, 761163, 556984, 591273, 4951, 515001, 83666, 155, 146233, 429200, 84892, 454983, 521777, 458723, 727745, 271110, 439079, 521029, 324857, 1592, 4552, 674, 177572, 372754, 37799, 346364, 4951, 1930, 37707, 438631, 897429, 537116, 634649, 558, 559, 537116, 331482, 8363, 50646, 396371, 414906, 810693, 82693, 4347, 466282, 10494, 18329, 81481, 347201, 361743, 787, 212716, 161, 11247, 36728, 138843, 400928, 762504, 116149, 511809, 280092, 578, 929170, 614934, 244786, 466282, 329865, 19913, 614409, 565426, 567609, 210577, 396535, 43949, 10096, 164558, 1022102, 376660, 81250, 9297, 508965, 1038198, 493922, 384677, 4348, 675, 674, 767, 12445, 12444, 913290, 315162, 545611, 13649, 745, 37799, 338967, 4232, 

Since we have 201 total entries but only 188 films with IDs, this must mean the remaining are TV shows or short films. We will separate the TV shows.

In [11]:
zero_search = 0 # for number of results with no ID
tv_title = []
tv_year = []

for show, year in zip(titles, release_years):
    response = search.movie(query=show, year=year)
    if (len(response['results'])) == 0:
        tv_title.append(show)
        tv_year.append(year)
        print(show, year, len(response['results']))
        zero_search += 1
print(" Zero Search: ", zero_search)
print(tv_title)
print(tv_year)

Rascal Does Not Dream of Bunny Girl Senpai 2018 0
DARLING in the FRANXX 2018 0
Black Mirror: Demon 79 2023 0
Black Mirror: Hang the DJ 2017 0
Black Mirror: Rachel, Jack and Ashley Too 2019 0
The Jinx: The Life and Deaths of Robert Durst 2015 0
The Undoing 2020 0
 Zero Search:  7
['Rascal Does Not Dream of Bunny Girl Senpai', 'DARLING in the FRANXX', 'Black Mirror: Demon 79', 'Black Mirror: Hang the DJ', 'Black Mirror: Rachel, Jack and Ashley Too', 'The Jinx: The Life and Deaths of Robert Durst', 'The Undoing']
[2018, 2018, 2023, 2017, 2019, 2015, 2020]


In [12]:
total_shows = 0
tv_id = []

for show, year in zip(tv_title, tv_year):
    response = search.tv(query = show, year = year)
    for value in response['results']:
        if value['name'] == show:
            tv_id.append(value['id'])
            total_shows += 1

print("Results count: ", total_shows)

Results count:  4


As a summary, we started with 201 entries. There are 188 films with ID. 7/13 of the missing entries are TV shows or episodes with only 4/7 of them having IDs. Thus, the remaining 6 are short films.

In [13]:
movie_info = []

for movie in movie_id:
    movie = tmdb.Movies(movie)
    response = movie.info()
    movie_info.append([movie.id, movie.title, movie.spoken_languages, movie.genres ,movie.runtime,
                      movie.release_date, movie.vote_count,  movie.vote_average])
    
print(len(movie_info))

188


In [14]:
tv_info = []

for show in tv_id:
    tv = tmdb.TV(show)
    response = tv.info()
    tv_info.append([tv.id, tv.name, tv.genres, 
                   tv.spoken_languages, tv.episode_run_time,
                   tv.first_air_date, tv.number_of_seasons, 
                   tv.number_of_episodes, tv.vote_count, tv.vote_average])
    
print(len(tv_info))

4


In [15]:
# Convert into dataframes and export as .csv files

movie_df = pd.DataFrame(movie_info, columns = ['ID', 'title', 'languages', 'genres','runtime',
                                               'release_date', 'vote_count', 'vote_average'])

movie_df.head()

,ID,title,languages,genres,runtime,release_date,vote_count,vote_average
0,496243,Parasite,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",133,2019-05-30,17972,8.500
1,419430,Get Out,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'id': 9648, 'name': 'Mystery'}, {'id': 53, '...",104,2017-02-24,17130,7.622
2,129,Spirited Away,"[{'english_name': 'Japanese', 'iso_639_1': 'ja...","[{'id': 16, 'name': 'Animation'}, {'id': 10751...",125,2001-07-20,16285,8.500
3,458723,Us,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",116,2019-03-14,7212,6.970
4,315635,Spider-Man: Homecoming,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",133,2017-07-05,21515,7.338


In [16]:
tv_df = pd.DataFrame(tv_info, columns = ['ID', 'title', 'genres', 'languages', 'ep_runtime',
                                        'first_air_date', 'total_seasons', 'total_eps', 'vote_count',
                                        'vote_average'])

tv_df.head()

,ID,title,genres,languages,ep_runtime,first_air_date,total_seasons,total_eps,vote_count,vote_average
0,82739,Rascal Does Not Dream of Bunny Girl Senpai,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'english_name': 'Japanese', 'iso_639_1': 'ja...",[24],2018-10-04,2,13,1177,8.567
1,76121,DARLING in the FRANXX,"[{'id': 16, 'name': 'Animation'}]","[{'english_name': 'Japanese', 'iso_639_1': 'ja...",[24],2018-01-13,1,24,1743,8.564
2,61929,The Jinx: The Life and Deaths of Robert Durst,"[{'id': 99, 'name': 'Documentary'}, {'id': 80,...","[{'english_name': 'English', 'iso_639_1': 'en'...",[],2015-02-08,2,12,430,7.991
3,83851,The Undoing,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...",[60],2020-10-25,1,6,691,7.415


In [17]:
# export into .csv files

movie_df.to_csv('watchedmovies.csv')
print("Exported movies succesfully!")

tv_df.to_csv('watchedTV.csv')
print("Exported TV shows successfully!")

Exported movies succesfully!
Exported TV shows successfully!


# Analyzing the overall data

In [18]:
# How many did I review?

q = """
    SELECT COUNT(*) FROM reviewed
    """
reviewed.to_sql('reviewed', connection, if_exists='replace', index=False)
result= pd.read_sql_query(q, connection)
print(f"Number of films reviewed: {result.iloc[0,0]}")


Number of films reviewed: 151


In [19]:
# How many did I like?

q3 = """
    SELECT COUNT(*) FROM liked_films
    """
liked_films.to_sql('liked_films', connection, if_exists='replace', index=False)
result = pd.read_sql_query(q3, connection)
print(f"Number of films liked: {result.iloc[0,0]}")

Number of films liked: 35


In [20]:
# How are the ratings spread?

q4 = """
    SELECT rating, COUNT(*) as number_of_films
    FROM watched
    GROUP BY Rating
    ORDER BY Rating
     """
watched.to_sql('watched', connection, if_exists='replace', index=False)
ratings_spread = pd.read_sql_query(q4, connection)
print(ratings_spread)

   Rating  number_of_films
0     NaN                6
1     0.5                1
2     1.0                3
3     2.0               11
4     2.5                6
5     3.0               32
6     3.5               33
7     4.0               45
8     4.5               32
9     5.0               32


In [21]:
# How many have I rewatched?

q5 = """
     SELECT Rewatch, COUNT(*)
     FROM watched
     GROUP BY Rewatch
     """

movies_rewatched = pd.read_sql_query(q5, connection)
print(movies_rewatched)

  Rewatch  COUNT(*)
0    None       177
1     Yes        24


# Analyzing films

In [22]:
films = pd.read_csv('watchedmovies.csv')

In [23]:
# How many minutes did I spend watching films?

q6 = """
    SELECT SUM(runtime)
    FROM films
     """

films.to_sql('films', connection, if_exists='replace', index=False)
minutes_watched = pd.read_sql_query(q6, connection)
print(f"Number of minutes watched: {minutes_watched.iloc[0,0]}")

Number of minutes watched: 21661


In [24]:
# How many times did each genre appear?

from collections import Counter
import ast

def extract_genres(obj):
    if isinstance(obj, str):
        return ast.literal_eval(obj)
    elif isinstance(obj, list):
        return obj
    else:
        return []


films['genres'] = films['genres'].apply(extract_genres)

genres_list = []
for idx, row in films.iterrows():
    title = row['title']
    for genre in row['genres']:
        genres_list.append((title, genre))
        
genres_df = pd.DataFrame(genres_list, columns=['title', 'genre'])

genres_df['title'] = genres_df['title'].astype(str)
genres_df['genre'] = genres_df['genre'].astype(str)

connection.execute("DROP TABLE IF EXISTS genres_table;")
connection.execute("""
                   CREATE TABLE genres_table (
                   title VARCHAR(255),
                   genre VARCHAR(255)
                );
                   """)

genres_df.to_sql('genres_table', connection, if_exists='append', index=False)

q7 = """
     SELECT genre, COUNT(*) as number_of_films
     FROM genres_table
     GROUP BY genre
     ORDER BY number_of_films DESC;
     """
genres = pd.read_sql_query(q7, connection)
print(genres)

                                     genre  number_of_films
0              {'id': 18, 'name': 'Drama'}               77
1             {'id': 35, 'name': 'Comedy'}               60
2          {'id': 12, 'name': 'Adventure'}               42
3         {'id': 10749, 'name': 'Romance'}               39
4           {'id': 53, 'name': 'Thriller'}               38
5             {'id': 27, 'name': 'Horror'}               35
6             {'id': 28, 'name': 'Action'}               27
7          {'id': 9648, 'name': 'Mystery'}               25
8          {'id': 16, 'name': 'Animation'}               25
9            {'id': 14, 'name': 'Fantasy'}               25
10             {'id': 80, 'name': 'Crime'}               24
11         {'id': 10751, 'name': 'Family'}               22
12  {'id': 878, 'name': 'Science Fiction'}               20
13          {'id': 10402, 'name': 'Music'}               13
14       {'id': 99, 'name': 'Documentary'}               10
15           {'id': 36, 'name': 'History

In [25]:
# What were the most popular languages?
def extract_languages(obj):
    if isinstance(obj, str):
        return ast.literal_eval(obj)
    elif isinstance(obj, list):
        return obj
    else:
        return []

films['languages'] = films['languages'].apply(extract_languages)

languages_list = []
for idx, row in films.iterrows():
    title = row['title']
    for language in row['languages']:
        languages_list.append((title, language))
        
languages_df = pd.DataFrame(languages_list, columns=['title', 'language_name'])

languages_df['title'] = languages_df['title'].astype(str)
languages_df['language_name'] = languages_df['language_name'].astype(str)

connection.execute("DROP TABLE IF EXISTS languages_table;")
connection.execute("""
                   CREATE TABLE languages_table (
                   title VARCHAR(255),
                   language_name VARCHAR(255)
                );
                   """)

languages_df.to_sql('languages_table', connection, if_exists='append', index=False)

q8 = """
     SELECT language_name, COUNT(*) as number_of_films
     FROM languages_table
     GROUP BY language_name
     ORDER BY number_of_films DESC;
     """
languages_spoken = pd.read_sql_query(q8, connection)
print(languages_spoken)

                                        language_name  number_of_films
0   {'english_name': 'English', 'iso_639_1': 'en',...              162
1   {'english_name': 'Japanese', 'iso_639_1': 'ja'...               18
2   {'english_name': 'Spanish', 'iso_639_1': 'es',...               16
3   {'english_name': 'Mandarin', 'iso_639_1': 'zh'...               11
4   {'english_name': 'Korean', 'iso_639_1': 'ko', ...               10
5   {'english_name': 'French', 'iso_639_1': 'fr', ...                9
6   {'english_name': 'German', 'iso_639_1': 'de', ...                7
7   {'english_name': 'Cantonese', 'iso_639_1': 'cn...                4
8   {'english_name': 'Russian', 'iso_639_1': 'ru',...                3
9   {'english_name': 'Romanian', 'iso_639_1': 'ro'...                3
10  {'english_name': 'Portuguese', 'iso_639_1': 'p...                3
11  {'english_name': 'Italian', 'iso_639_1': 'it',...                3
12  {'english_name': 'Tagalog', 'iso_639_1': 'tl',...                2
13  {'